In [ ]:
import re

In [ ]:
text = R"""
#include "pybind11/pybind11.h"

#include "drake/bindings/pydrake/documentation_pybind.h"
#include "drake/bindings/pydrake/pydrake_pybind.h"
#include "drake/multibody/parsers/package_map.h"

#define D(...) DOC(drake, multibody, parsing, __VA_ARGS__)

namespace drake {
namespace pydrake {

PYBIND11_MODULE(parsers, m) {
  using drake::multibody::parsing::PackageMap;

  m.doc() = "Tools for loading robots from various files";

  py::class_<PackageMap>(m, "PackageMap", D(PackageMap))
    .def(py::init<>(), D(PackageMap, PackageMap))
    .def("Add", &PackageMap::Add, D(PackageMap, Add))
    .def("Contains", &PackageMap::Contains, D(PackageMap, Contains))
    .def("size", &PackageMap::size, D(PackageMap, size))
    .def("GetPath", &PackageMap::GetPath, D(PackageMap, GetPath))
    .def("PopulateFromFolder", &PackageMap::PopulateFromFolder,
         D(PackageMap, PopulateFromFolder))
    .def("PopulateFromEnvironment", &PackageMap::PopulateFromEnvironment,
         D(PackageMap, PopulateFromEnvironment))
    .def("PopulateUpstreamToDrake", &PackageMap::PopulateUpstreamToDrake,
         D(PackageMap, PopulateUpstreamToDrake));
}

}  // namespace pydrake
}  // namespace drake
"""

In [ ]:
pattern = r"#define D\(...\) DOC\((.*?), __VA_ARGS__\)"

def sub(m):
    pieces = m.group(1).split(", ")
    return "auto& doc = pydrake_doc." + ".".join(pieces) + ";"

p2 = r"\bD\((.*?)\)"

def sub2(m):
    pieces = m.group(1).split(", ")
    suffix = ""
    try:
        index = int(pieces[-1])
        pieces = pieces[:-1]
        suffix = "_{}".format(index)
    except ValueError:
        pass
    return "doc." + ".".join(pieces) + ".doc" + suffix

tnew = re.sub(pattern, sub, text, re.M)
tnew = re.sub(p2, sub2, tnew, re.M)

print(tnew)

In [ ]:
m_.group(1)

In [ ]:
ms

In [ ]:
?re.sub